## 환경 셋팅

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

In [3]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [4]:
# 프롬프트 템플릿 설정 및 체인 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 Python 전문가 입니다."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9, 
)
chain = prompt | llm

In [5]:
query_1 = "LangChain을 5문장으로 설명해 주세요."
query_2 = "주요 기능 5가지만 설명해 주세요."

## RunnableWithMessageHistory 구현

### 1\) In Memory 활용

In [6]:
# 메모리 기반 히스토리 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 히스토리 가져오기
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [7]:
# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID를 사용하여체인 실행
response = chain_with_history.invoke(
    {"input": query_1},
    config={"configurable": {"session_id": "user_1"}}
)
print(f"답변:\n{response.content}")

답변:
LangChain은 자연어 처리 애플리케이션 개발을 돕는 오픈소스 프레임워크입니다. 주로 대형 언어 모델(LLM)과의 상호작용을 쉽게 만들기 위해 설계되었습니다. 데이터 연결, 체인 구성, 에이전트 설계 등 다양한 기능을 제공합니다. 이를 통해 복잡한 작업을 여러 단계로 나누어 처리할 수 있습니다. Python을 포함한 여러 언어에서 활용 가능하며, 확장성과 유연성이 뛰어납니다.


In [8]:
# 대화 히스토리 출력 
history = get_session_history("user_1")
pprint(history.messages)

[HumanMessage(content='LangChain을 5문장으로 설명해 주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LangChain은 자연어 처리 애플리케이션 개발을 돕는 오픈소스 프레임워크입니다. 주로 대형 언어 모델(LLM)과의 상호작용을 쉽게 만들기 위해 설계되었습니다. 데이터 연결, 체인 구성, 에이전트 설계 등 다양한 기능을 제공합니다. 이를 통해 복잡한 작업을 여러 단계로 나누어 처리할 수 있습니다. Python을 포함한 여러 언어에서 활용 가능하며, 확장성과 유연성이 뛰어납니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 30, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-Bd9cxYNYRFhr6T1IW1tO321llbiOJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3e474ebc-ec79-480b-92d6-aebad5fe6f81-0', usage_metadata={'input_tokens': 30, 'output_tokens': 112, 'total_tokens':

In [9]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {"input": query_2},
    config={"configurable": {"session_id": "user_1"}}
)
print(f"답변:\n{response.content}")

답변:
LangChain의 주요 기능 5가지는 다음과 같습니다:

1. **체인(Chains) 구성**: 여러 개의 언어 모델 호출이나 데이터 처리 단계를 순차적 또는 병렬로 연결해 복잡한 워크플로우를 쉽게 구현할 수 있습니다.  
2. **프롬프트 템플릿(Prompt Templates)**: 재사용 가능한 프롬프트를 생성하고 관리하여 일관된 입력을 언어 모델에 전달할 수 있습니다.  
3. **에이전트(Agents)**: 외부 도구나 API와 상호작용하며, 상황에 맞게 적절한 행동을 선택해 자동화된 의사결정을 수행합니다.  
4. **메모리(Memory)**: 대화형 애플리케이션에서 이전 대화 내용을 저장하고 활용하여 문맥을 유지할 수 있습니다.  
5. **데이터 연결(Data Connectors)**: 데이터베이스, 문서, API 등 다양한 외부 데이터 소스와 연동해 정보를 가져오고 처리할 수 있습니다.


In [10]:
# 대화 히스토리 출력 
history = get_session_history("user_1")
pprint(history.messages)

[HumanMessage(content='LangChain을 5문장으로 설명해 주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LangChain은 자연어 처리 애플리케이션 개발을 돕는 오픈소스 프레임워크입니다. 주로 대형 언어 모델(LLM)과의 상호작용을 쉽게 만들기 위해 설계되었습니다. 데이터 연결, 체인 구성, 에이전트 설계 등 다양한 기능을 제공합니다. 이를 통해 복잡한 작업을 여러 단계로 나누어 처리할 수 있습니다. Python을 포함한 여러 언어에서 활용 가능하며, 확장성과 유연성이 뛰어납니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 30, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-Bd9cxYNYRFhr6T1IW1tO321llbiOJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3e474ebc-ec79-480b-92d6-aebad5fe6f81-0', usage_metadata={'input_tokens': 30, 'output_tokens': 112, 'total_tokens':

### 2\) SQLite 활용

In [11]:
import json
import sqlite3

In [12]:
class SQLiteChatMessageHistory(BaseChatMessageHistory):
    """ 
    SQLite 데이터베이스를 사용하여 챗봇 히스토리를 저장하는 클래스

    Attributes:
        session_id (str): 세션 ID
        db_path (str): SQLite 데이터베이스 파일 경로
    """
    def __init__(self, session_id: str, db_path: str = "chat_history.db"):
        self.session_id = session_id
        self.db_path = db_path
        self._create_tables()
    
    def _create_tables(self):
        """데이터베이스 테이블 생성"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 메시지 테이블 생성
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS messages (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT,
                message_type TEXT,
                content TEXT,
                metadata TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        conn.commit()
        conn.close()
    
    def add_message(self, message: BaseMessage) -> None:
        """단일 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO messages (session_id, message_type, content, metadata)
            VALUES (?, ?, ?, ?)
        """, (
            self.session_id,
            message.__class__.__name__,
            message.content,
            json.dumps(message.additional_kwargs)
        ))
        
        conn.commit()
        conn.close()
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """여러 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for message in messages:
            cursor.execute("""
                INSERT INTO messages (session_id, message_type, content, metadata)
                VALUES (?, ?, ?, ?)
            """, (
                self.session_id,
                message.__class__.__name__,
                message.content,
                json.dumps(message.additional_kwargs)
            ))
        
        conn.commit()
        conn.close()
    
    def clear(self) -> None:
        """세션의 모든 메시지 삭제"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            DELETE FROM messages WHERE session_id = ?
        """, (self.session_id,))
        
        conn.commit()
        conn.close()
    
    @property
    def messages(self) -> List[BaseMessage]:
        """저장된 메시지 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT message_type, content, metadata
            FROM messages 
            WHERE session_id = ?
            ORDER BY created_at
        """, (self.session_id,))
        
        messages = []
        for row in cursor.fetchall():
            message_type, content, metadata = row
            if message_type == "HumanMessage":
                message = HumanMessage(content=content)
            else:
                message = AIMessage(content=content)
            
            if metadata:
                message.additional_kwargs = json.loads(metadata)
            
            messages.append(message)
        
        conn.close()
        return messages
    

# 세션 ID로 히스토리 가져오기
def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    return SQLiteChatMessageHistory(session_id=session_id)

In [13]:
# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여체인 실행
response = chain_with_history.invoke(
    {"input": query_1},
    config={"configurable": {"session_id": "user_2"}}
)
print(f"답변:\n{response.content}")

답변:
LangChain은 자연어 처리 애플리케이션 개발을 돕는 오픈소스 프레임워크입니다. 주로 대형 언어 모델(LLM)과의 인터페이스를 간편하게 만들어 줍니다. 데이터 연결, 체인 구성, 메모리 관리 등 다양한 기능을 제공합니다. 이를 통해 복잡한 작업을 여러 단계로 나누어 처리할 수 있습니다. Python을 포함한 여러 프로그래밍 언어에서 활용 가능합니다.


In [14]:
# 대화 히스토리 출력 
history = get_chat_history("user_2")
pprint(history.messages)

[HumanMessage(content='LangChain을 5문장으로 설명해 주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LangChain은 자연어 처리 애플리케이션 개발을 돕는 오픈소스 프레임워크입니다. 주로 대형 언어 모델(LLM)과의 인터페이스를 간편하게 만들어 줍니다. 데이터 연결, 체인 구성, 메모리 관리 등 다양한 기능을 제공합니다. 이를 통해 복잡한 작업을 여러 단계로 나누어 처리할 수 있습니다. Python을 포함한 여러 프로그래밍 언어에서 활용 가능합니다.', additional_kwargs={'refusal': None}, response_metadata={})]


In [15]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {"input": query_2},
    config={"configurable": {"session_id": "user_2"}}
)
print(f"답변:\n{response.content}")

답변:
LangChain의 주요 기능 5가지는 다음과 같습니다:

1. **체인(Chains) 구성**: 여러 개의 언어 모델 호출이나 데이터 처리 단계를 순차적으로 연결해 복잡한 작업을 자동화할 수 있습니다.  
2. **프롬프트 템플릿(Prompt Templates)**: 동적인 입력값을 받아 다양한 상황에 맞는 프롬프트를 쉽게 생성하고 관리할 수 있습니다.  
3. **메모리(Memory) 관리**: 대화형 애플리케이션에서 이전 대화 내용을 저장하고 참조하여 문맥을 유지할 수 있습니다.  
4. **데이터 연결(Data Connectors)**: 외부 데이터베이스, API, 문서 등 다양한 데이터 소스와 연동하여 정보를 가져오고 활용할 수 있습니다.  
5. **에이전트(Agents)**: 사용자의 목표에 맞춰 여러 도구와 모델을 조합해 자율적으로 문제를 해결하는 기능을 제공합니다.


In [16]:
# 대화 히스토리 출력 
history = get_chat_history("user_2")
pprint(history.messages)

[HumanMessage(content='LangChain을 5문장으로 설명해 주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LangChain은 자연어 처리 애플리케이션 개발을 돕는 오픈소스 프레임워크입니다. 주로 대형 언어 모델(LLM)과의 인터페이스를 간편하게 만들어 줍니다. 데이터 연결, 체인 구성, 메모리 관리 등 다양한 기능을 제공합니다. 이를 통해 복잡한 작업을 여러 단계로 나누어 처리할 수 있습니다. Python을 포함한 여러 프로그래밍 언어에서 활용 가능합니다.', additional_kwargs={'refusal': None}, response_metadata={}),
 HumanMessage(content='주요 기능 5가지만 설명해 주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LangChain의 주요 기능 5가지는 다음과 같습니다:\n\n1. **체인(Chains) 구성**: 여러 개의 언어 모델 호출이나 데이터 처리 단계를 순차적으로 연결해 복잡한 작업을 자동화할 수 있습니다.  \n2. **프롬프트 템플릿(Prompt Templates)**: 동적인 입력값을 받아 다양한 상황에 맞는 프롬프트를 쉽게 생성하고 관리할 수 있습니다.  \n3. **메모리(Memory) 관리**: 대화형 애플리케이션에서 이전 대화 내용을 저장하고 참조하여 문맥을 유지할 수 있습니다.  \n4. **데이터 연결(Data Connectors)**: 외부 데이터베이스, API, 문서 등 다양한 데이터 소스와 연동하여 정보를 가져오고 활용할 수 있습니다.  \n5. **에이전트(Agents)**: 사용자의 목표에 맞춰 여러 도구와 모델을 조합해 자율적으로 문제를 해결하는 기능을 제공합니다.', additional_kwargs={'refusal': None}, response_metadata={})]
